In [1]:
import pandas
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

from toolz import pipe
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from model import DynamicsLearner

dataset_path = "../data/m100"
client = M100DataClient(dataset_path)



In [ ]:
"""Cell purpose: pull desired features from plugins"""

#vertiv plugin collects data for AC cooling 
vertiv_df = client.query(["Supply_Air_Temperature", "Return_Air_Temperature"],
                  columns=['timestamp','value'],
                  year_month=["22-02"])
vertiv_df = vertiv_df.rename(columns = {"value": "air_temp" })



#ipmi pluging collects physical/hardware properties of nodes

ipmi_df = client.query(["ambient", "gpu0_core_temp", "p0_core0_temp"], #starting with just one core for each, work out indexing deal later
                  columns=['timestamp', "node", 'value'],
                  year_month=["22-02"])
ipmi_df = ipmi_df.rename(columns = {"value": "ambient_temp" }).iloc[:101000]

# #ganglia collects metrics on utilization of hardware
# ganglia_df = client.query(["load_fifteen"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])
#logics collects power consumption data from equipment and devices
# logics_df = client.query(["Tot"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

#weather data
weather_df = client.query(["temp"],
                  columns=['timestamp', "node", 'value'],
                  year_month=["22-02"])



Retrieving data of type: float
Retrieving data of type: float
Retrieving data of type: int32


In [ ]:
# dfs = [weather_df, schneider_df, logics_df, ganglia_df, ipmi_df, vertiv_df]
dfs = [ipmi_df, vertiv_df, weather_df]
for d in dfs:
    print(d.shape)
    print(d.head(3))

In [ ]:
"""Cell purpose: preprocess and/or merge data as desired"""
df = dfs[0].join(dfs[1:])
# print(merged_df["timestamp"].astype(np.int64) * 10e-9)
# merge_df["timestamp"] = merge_df["timestamp"].apply(lambda ts: ts.as_unit("ns"))
merged_df.shape
train_df = merged_df[:-100]
test_df = merged_df[-100:]

In [ ]:
"""Cell purpose: train"""
model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)


timesteps = [t for t in range(preds.shape[0])]
colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
plt.scatter(preds, labels, c=["blue", "red"])

plt.legend(loc="upper left")
plt.xlabel("Time")
plt.ylabel("Ambient temperature")
plt.grid()
plt.show()



In [ ]:
""
